In [21]:
from torchvision import models, transforms
import torch
from torch import nn
from torchinfo import summary

In [22]:
weights = models.ConvNeXt_Small_Weights.IMAGENET1K_V1
model = models.convnext_small(weights=weights)

In [23]:
new_first_layer = nn.Conv2d(
    in_channels=8,
    out_channels=96,   
    kernel_size=4,
    stride=4,
    padding=0
)

with torch.no_grad():
    new_first_layer.weight[:, :3] = model.features[0][0].weight.clone()
    new_first_layer.weight[:, 3:] = torch.mean(model.features[0][0].weight, dim=1, keepdim=True).repeat(1, 5, 1, 1)

model.features[0][0] = new_first_layer
model.classifier[2] = nn.Linear(in_features=768, out_features=1, bias=True)

In [24]:
summary(model)

Layer (type:depth-idx)                        Param #
ConvNeXt                                      --
├─Sequential: 1-1                             --
│    └─Conv2dNormActivation: 2-1              --
│    │    └─Conv2d: 3-1                       12,384
│    │    └─LayerNorm2d: 3-2                  192
│    └─Sequential: 2-2                        --
│    │    └─CNBlock: 3-3                      79,296
│    │    └─CNBlock: 3-4                      79,296
│    │    └─CNBlock: 3-5                      79,296
│    └─Sequential: 2-3                        --
│    │    └─LayerNorm2d: 3-6                  192
│    │    └─Conv2d: 3-7                       73,920
│    └─Sequential: 2-4                        --
│    │    └─CNBlock: 3-8                      306,048
│    │    └─CNBlock: 3-9                      306,048
│    │    └─CNBlock: 3-10                     306,048
│    └─Sequential: 2-5                        --
│    │    └─LayerNorm2d: 3-11                 384
│    │    └─Conv2d: 3-12  

In [34]:
print(model.features[1][1].block[0].weight)

Parameter containing:
tensor([[[[-2.9842e-03, -2.9003e-03, -5.2862e-03,  ..., -6.8963e-03,
           -3.1104e-03, -6.1358e-04],
          [-1.9168e-03,  7.0077e-04, -2.9985e-04,  ..., -6.8603e-03,
           -5.7568e-03, -4.3743e-03],
          [-3.6474e-05,  2.7816e-03,  2.2459e-02,  ..., -1.6962e-02,
           -5.5806e-03, -7.3486e-03],
          ...,
          [ 3.8451e-03,  8.0612e-03,  1.4248e-02,  ..., -1.9534e-02,
           -4.9930e-03, -5.7488e-03],
          [-3.9446e-03,  2.1202e-03,  1.2755e-03,  ..., -1.2831e-02,
           -8.4448e-03, -2.5062e-03],
          [ 1.3798e-03,  1.5740e-03, -6.2430e-03,  ..., -4.0893e-03,
           -6.3052e-03, -7.4488e-03]]],


        [[[-3.1285e-04,  3.6523e-04, -1.5173e-03,  ..., -2.0336e-03,
            1.8035e-03,  1.9020e-04],
          [-2.5782e-04,  1.0236e-03, -9.8235e-04,  ..., -1.7132e-03,
           -1.2277e-03, -1.6443e-03],
          [-6.4671e-04, -7.0869e-04, -4.2115e-03,  ..., -4.6993e-04,
           -7.4760e-04,  9.6017e-0